In [1]:
from google.cloud import aiplatform, aiplatform_v1, storage, run_v2
import joblib
import tempfile
import os
import requests
import time
from google.auth import default

In [2]:
project_id = 'deeplearning-sahil'
location = 'us-central1'
aiplatform.init(project=project_id, location=location)

In [3]:
client = aiplatform_v1.ModelServiceClient(
        client_options={"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    )

I0000 00:00:1757802530.824399 54743774 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [4]:
# models = aiplatform.Model.list(filter=f"display_name={model_name}")
# models

In [5]:
# credentials, _ = default()

In [6]:
# credentials.client_secret

In [7]:
# credentials.client_id

In [8]:
model_name = 'Iris-Classifier-XGBoost'
request = {
        "parent": f"projects/{project_id}/locations/{location}",
        "filter": f"display_name={model_name}"
    }

models = list(client.list_models(request=request))
blessed_model = None

In [9]:
models

[name: "projects/57434141298/locations/us-central1/models/2672742342821150720"
 version_id: "1"
 version_aliases: "default"
 version_create_time {
   seconds: 1746422998
   nanos: 130806000
 }
 version_update_time {
   seconds: 1746424646
   nanos: 66237000
 }
 display_name: "Iris-Classifier-XGBoost"
 predict_schemata {
   instance_schema_uri: "https://storage.googleapis.com/caip-tenant-4db17081-7d07-424f-8df2-cae4e3bebe9d/schemata/predict_instance.yaml?GoogleAccessId=service-57434141298@gcp-sa-aiplatform.iam.gserviceaccount.com&Expires=1757806133&Signature=VK37l48UEL%252FGsZ6vmJnUKOB3cKYXT2wrxrLdArqlQWytCtS%252Beh3ReN74V%252BAVq10312hiWkDotoFDY2xsiUTI%252BNkzODIOkSgATNCaFyx22uYe2zBFA%252BxGMoit9n8eXzez8Rx57Eno4gC6pyD1jI5mowZ7BU5HTZRlR5Ey1KTTpmmCrLjw0lyUi8sIwTXbVDLrbaPF%252FMH4cWQ6vw8voZ97btIBP9XtwMJ5FdQ2U2LizRmTU9rbknGz3Q9AeAmbyAcY3JIRcTLUOXUA2SlbEqmP5UP831VVRc6%252BwlMzRDo7FBZCt1ku8mxpsuQ4vavrcxtUxVVTKuMLUgs6uL1%252BO%252Fp59A%253D%253D"
   prediction_schema_uri: "https://storage.goo

In [10]:
print(f"Found {len(models)} model versions with name {model_name}")

Found 1 model versions with name Iris-Classifier-XGBoost


In [11]:
for parent_model in models:
    print(f"Checking parent model: {parent_model.name}")
    
    # List all versions of this model
    versions_request = {"name": parent_model.name}
    versions = list(client.list_model_versions(request=versions_request))
    
    print(f"Found {len(versions)} versions for this model")
    
    for version in versions:
        print(f"Version {version.version_id}: Aliases = {list(version.version_aliases)}")
        if "blessed" in version.version_aliases:
            blessed_model = version
            print(f"Found blessed version: {version.version_id}")
            break
    
    if blessed_model:
        break

Checking parent model: projects/57434141298/locations/us-central1/models/2672742342821150720
Found 24 versions for this model
Version 1: Aliases = ['default']
Version 2: Aliases = []
Version 3: Aliases = []
Version 4: Aliases = []
Version 5: Aliases = []
Version 6: Aliases = []
Version 7: Aliases = []
Version 8: Aliases = []
Version 9: Aliases = []
Version 10: Aliases = []
Version 11: Aliases = []
Version 12: Aliases = []
Version 13: Aliases = []
Version 14: Aliases = []
Version 15: Aliases = []
Version 16: Aliases = []
Version 17: Aliases = []
Version 18: Aliases = []
Version 19: Aliases = []
Version 20: Aliases = []
Version 21: Aliases = []
Version 22: Aliases = []
Version 23: Aliases = []
Version 24: Aliases = ['blessed']
Found blessed version: 24


In [12]:
blessed_model

name: "projects/57434141298/locations/us-central1/models/2672742342821150720@24"
version_id: "24"
version_aliases: "blessed"
version_create_time {
  seconds: 1757800066
  nanos: 832425000
}
version_update_time {
  seconds: 1757800222
  nanos: 956698000
}
display_name: "Iris-Classifier-XGBoost"
predict_schemata {
  instance_schema_uri: "https://storage.googleapis.com/caip-tenant-46ea112b-0e41-4566-be6b-6d7ddd0712be/schemata/predict_instance.yaml?GoogleAccessId=service-57434141298@gcp-sa-aiplatform.iam.gserviceaccount.com&Expires=1757806134&Signature=KbCCFUJcdymZ9gRtiTTR2zB31yL7VE0XB6VGz0eFeODLrmU4jsKaxXVefyupnxFTG7ONMqtQM2flqvuLwhAq4D28sAP3WzjbiTnit4Jzf5z%252B%252Fl7vBgnLbPAxR7mCPuM5S0nJ%252FXiNV4Fzn%252Fv6otVte4wIswdT9cp2jevD%252FHX2dWsbAor750gGyZj%252Bvg1YS4FID%252Bzmud1%252F8Gb0X9Dzm4SmQNgc%252FzyEe7c5OHLJIFo%252FnMsOIU8kRIV%252B3sefeU9rnQscspuXxInqe%252BB7ksfZaLY8X%252BCxOkX%252Fm4SEqm8V8gkwyN83RSWK55wU6l%252FuKHynANMsnmoHmdL2mSZeRUe62VYhvA%253D%253D"
  prediction_schema_uri: "https

In [13]:
gcs_uri = blessed_model.artifact_uri
gcs_uri

'gs://sb-vertex/pipeline_root/57434141298/pipeline-iris-20250913174151/choose-best-model_1726793559861886976/best_model'

In [14]:
bucket_name = gcs_uri.replace('gs://', '').split('/')[0]
bucket_name

'sb-vertex'

In [15]:
model_path = '/'.join(gcs_uri.replace('gs://', '').split('/')[1:])
model_path

'pipeline_root/57434141298/pipeline-iris-20250913174151/choose-best-model_1726793559861886976/best_model'

In [16]:
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

In [17]:
model_blob_path = f"{model_path}/model.joblib"
blob = bucket.blob(model_blob_path)

In [18]:
blob

<Blob: sb-vertex, pipeline_root/57434141298/pipeline-iris-20250913174151/choose-best-model_1726793559861886976/best_model/model.joblib, None>

In [19]:
with tempfile.NamedTemporaryFile(suffix='.joblib', delete=False) as temp_file:
    blob.download_to_filename(temp_file.name)
    local_model_path = temp_file.name

print(f"Downloaded model to: {local_model_path}")

Downloaded model to: /var/folders/m0/zbb6htc128l81r3p8lhs13xm0000gn/T/tmpb4fe1gb7.joblib


In [20]:
try:
    model_obj = joblib.load(local_model_path)
    print(f"Model type: {type(model_obj)}")
    print(f"Model validation successful")
except Exception as e:
    os.unlink(local_model_path)
    raise ValueError(f"Model validation failed: {e}")

Model type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Model validation successful


/Users/shlba/Desktop/Docs/Study/code/ml_pipelines_kfp/.venv/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/shlba/Desktop/Docs/Study/code/ml_pipelines_kfp/.venv/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [21]:
model_name

'Iris-Classifier-XGBoost'

In [22]:
service_name=f"{model_name.lower().replace('_', '-')}-service"
service_name

'iris-classifier-xgboost-service'

In [23]:
# 4. Copy model to standard deployment location
deployment_model_path = f"deployed-models/{service_name}/model.joblib"
deployment_blob = bucket.blob(deployment_model_path)

print(f"Copying model to deployment location: gs://{bucket_name}/{deployment_model_path}")
deployment_blob.upload_from_filename(local_model_path)

model_gcs_path = f"gs://{bucket_name}/{deployment_model_path}"
print(f"Model available at: {model_gcs_path}")

Copying model to deployment location: gs://sb-vertex/deployed-models/iris-classifier-xgboost-service/model.joblib
Model available at: gs://sb-vertex/deployed-models/iris-classifier-xgboost-service/model.joblib


In [24]:
run_client = run_v2.ServicesClient()

I0000 00:00:1757802540.493297 54743774 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [25]:
generic_image =  "us-docker.pkg.dev/deeplearning-sahil/sahil-experiment-docker-images/fastapi-ml-generic:latest"
generic_image

'us-docker.pkg.dev/deeplearning-sahil/sahil-experiment-docker-images/fastapi-ml-generic:latest'

In [26]:
service_config = {
    "parent": f"projects/{project_id}/locations/{location}",
    "service_id": service_name,
    "service": {
        "template": {
            "containers": [{
                "image": generic_image,
                "ports": [{"container_port": 8080}],
                "resources": {
                    "limits": {
                        "memory": "2Gi",
                        "cpu": "2"
                    }
                },
                "env": [
                    {"name": "MODEL_GCS_PATH", "value": model_gcs_path},
                    {"name": "MODEL_NAME", "value": model_name},
                    {"name": "GOOGLE_CLOUD_PROJECT", "value": project_id}
                ]
            }],
            "scaling": {
                "min_instance_count": 0,
                "max_instance_count": 10
            },
            "service_account": f"kfp-mlops@{project_id}.iam.gserviceaccount.com"
        },
        "traffic": [{"percent": 100, "type": "TRAFFIC_TARGET_ALLOCATION_TYPE_LATEST"}]
    }
}


In [27]:
try:
    existing_service = run_client.get_service(
        name=f"projects/{project_id}/locations/{location}/services/{service_name}"
    )
    print(f"Service {service_name} already exists, updating...")
    
    # Update existing service
    update_service = service_config["service"]
    update_service["name"] = existing_service.name
    
    operation = run_client.update_service(service=update_service)
    result = operation.result(timeout=600)
    
except Exception as get_error:
    print(f"Service doesn't exist, creating new one: {get_error}")
    # Create new service
    operation = run_client.create_service(request=service_config)
    result = operation.result(timeout=600)

service_url = result.uri
print(f"Service deployed successfully to: {service_url}")

Service iris-classifier-xgboost-service already exists, updating...
Service deployed successfully to: https://iris-classifier-xgboost-service-zoxyfmo73q-uc.a.run.app


In [34]:
print("Testing deployment...")
time.sleep(1)  # Wait for service to be ready

test_payload = {
    "instances": [
        {"SepalLengthCm": 5.1, "SepalWidthCm": 3.5, "PetalLengthCm": 1.4, "PetalWidthCm": 0.2}
    ]
}

Testing deployment...


In [30]:
# # Ensure IAM policy allows unauthenticated access
# from google.cloud.run_v2 import SetIamPolicyRequest
# from google.iam.v1 import policy_pb2, binding_pb2

# # Create policy to allow public access
# policy = policy_pb2.Policy()
# binding = binding_pb2.Binding()
# binding.role = "roles/run.invoker"
# binding.members.append("allUsers")
# policy.bindings.append(binding)

# # Apply the policy
# iam_request = SetIamPolicyRequest(
#     resource=result.name,
#     policy=policy
# )
# run_client.set_iam_policy(request=iam_request)

ImportError: cannot import name 'SetIamPolicyRequest' from 'google.cloud.run_v2' (/Users/shlba/Desktop/Docs/Study/code/ml_pipelines_kfp/.venv/lib/python3.10/site-packages/google/cloud/run_v2/__init__.py)

In [32]:
# Ensure IAM policy allows unauthenticated access
from google.cloud.run_v2 import ServicesClient
from google.iam.v1 import iam_policy_pb2
from google.iam.v1.iam_policy_pb2 import SetIamPolicyRequest
from google.iam.v1 import policy_pb2

# Initialize the client
run_client = ServicesClient()

# Create policy to allow public access
policy = policy_pb2.Policy()
binding = policy_pb2.Binding()
binding.role = "roles/run.invoker"
binding.members.append("allUsers")
policy.bindings.append(binding)

# Apply the policy
iam_request = SetIamPolicyRequest(
    resource=result.name,  # This should be the full resource name
    policy=policy
)
run_client.set_iam_policy(request=iam_request)

I0000 00:00:1757802667.945853 54743774 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


version: 1
bindings {
  role: "roles/run.invoker"
  members: "allUsers"
}
etag: "\007\006>\266S\246\273\263"

In [35]:
try:
    # Test health endpoint first
    health_response = requests.get(f"{service_url}/health", timeout=30)
    print(f"Health check status: {health_response.status_code}")
    if health_response.status_code == 200:
        print(f"Health check response: {health_response.json()}")
    
    # Test prediction endpoint
    response = requests.post(
        f"{service_url}/predict", 
        json=test_payload,
        timeout=30
    )
    if response.status_code == 200:
        print("Deployment test successful!")
        print(f"Prediction: {response.json()}")
    else:
        print(f"Prediction test failed: {response.status_code} - {response.text}")
        
except Exception as test_e:
    print(f"Test request failed: {test_e}")

Health check status: 200
Health check response: {'status': 'healthy', 'model_type': "<class 'sklearn.ensemble._forest.RandomForestClassifier'>", 'model_loaded': True}
Deployment test successful!
Prediction: {'predictions': [{'prediction': 2}]}
